# chord finder
reference: https://github.com/davidparsson/transposer/blob/master/transposer.py

In [104]:
#https://github.com/davidparsson/transposer
# encoding: utf-8
"""
transposer.py
See README.md for information.
Created by David Pärsson on 2011-08-13.
Excessively doctested due to education.
"""

import sys
import os
import re, getopt
import pandas as pd

key_list = [('A',), ('A#', 'Bb'), ('B',), ('C',), ('C#', 'Db'), ('D',),
            ('D#', 'Eb'), ('E',), ('F',), ('F#', 'Gb'), ('G',), ('G#', 'Ab')]

sharp_flat = ['#', 'b']
sharp_flat_preferences = {
    'A' : '#',
    'A#': 'b',
	'Bb': 'b',
	'B' : '#',
	'C' : 'b',
	'C#': 'b',
	'Db': 'b',
	'D' : '#',
	'D#': 'b',
	'Eb': 'b',
	'E' : '#',
	'F' : 'b',
	'F#': '#',
	'Gb': '#',
	'G' : '#',
	'G#': 'b',
	'Ab': 'b',
	}

key_regex = re.compile(r"[ABCDEFG][#b]?")

def get_index_from_key(source_key):
	"""Gets the internal index of a key
	>>> get_index_from_key('Bb')
	1
	"""
	for key_names in key_list:
		if source_key in key_names:
			return key_list.index(key_names)
	raise Exception("Invalid key: %s" % source_key)

def get_key_from_index(index, to_key):
	"""Gets the key at the given internal index.
	Sharp or flat depends on the target key.
	>>> get_key_from_index(1, 'Eb')
	'Bb'
	"""
	key_names = key_list[index % len(key_list)]
	if len(key_names) > 1:
		sharp_or_flat = sharp_flat.index(sharp_flat_preferences[to_key])
		return key_names[sharp_or_flat]
	return key_names[0]

def get_transponation_steps(source_key, target_key):
	"""Gets the number of half tones to transpose
	>>> get_transponation_steps('D', 'C')
	-2
	"""
	source_index = get_index_from_key(source_key)
	target_index = get_index_from_key(target_key)
	return target_index - source_index



	
def recursive_line_transpose(source_line, source_chords, direction, to_key):
	if not source_chords or not source_line:
		return source_line
	source_chord = source_chords.pop(0)
	chord_index = source_line.find(source_chord)
	after_chord_index = chord_index + len(source_chord)
	
	return source_line[:chord_index] + \
		   transpose(source_chord, direction, to_key) + \
		   recursive_line_transpose(source_line[after_chord_index:], source_chords, direction, to_key)


def transpose(source_chord, direction, to_key):
	"""Transposes a chord a number of half tones.
	Sharp or flat depends on target key.
	>>> transpose('C', 3, 'Bb')
	'Eb'
	"""
	source_index = get_index_from_key(source_chord)
	return get_key_from_index(source_index + direction, to_key)


def transpose_line(source_line, direction, to_key):
	"""Transposes a line a number of keys if it starts with a pipe. Examples:
	>>> transpose_line('| A | A# | Bb | C#m7/F# |', -2, 'C')
	'| G | Ab | Ab | Bm7/E |'
	Different keys will be sharp or flat depending on target key.
	>>> transpose_line('| A | A# | Bb | C#m7/F# |', -2, 'D')
	'| G | G# | G# | Bm7/E |'
	It will use the more common key if sharp/flat, for example F# instead of Gb.
	>>> transpose_line('| Gb |', 0, 'Gb')
	'| F# |'
	Lines not starting with pipe will not be transposed
	>>> transpose_line('A | Bb |', -2, 'C')
	'A | Bb |'
	"""
	if source_line[0] != '|':
		return source_line
	source_chords = key_regex.findall(source_line)
	return recursive_line_transpose(source_line, source_chords, direction, to_key)

In [113]:
import pandas as pd

# changed 12 notes
before = '| C | D | Em | Bm | C#m7 | G | Am | D |'

key_regex_check = re.compile(r"[ABCDEFG][#bm]?")
chord_result = []
for i in range(1, 13):
    changed = transpose_line(before, i, 'B')
    changed2 = key_regex_check.findall(changed)
    chord_result.append(changed2)
    
changed_df = pd.DataFrame(chord_result)
changed_df

,0,1,2,3,4,5,6,7
0,C#,D#,Fm,Cm,Dm,G#,A#,D#
1,D,E,F#,C#,D#,A,Bm,E
2,D#,F,Gm,Dm,Em,A#,Cm,F
3,E,F#,G#,D#,Fm,B,C#,F#
4,F,G,Am,Em,F#,C,Dm,G
5,F#,G#,A#,Fm,Gm,C#,D#,G#
6,G,A,Bm,F#,G#,D,Em,A
7,G#,A#,Cm,Gm,Am,D#,Fm,A#
8,A,B,C#,G#,A#,E,F#,B
9,A#,C,Dm,Am,Bm,F,Gm,C


In [106]:
h = '| C | Dm | Em | Bm | Cm | Fm | Am | D |'
h_col = key_regex_check.findall(h)
harmony_result = pd.DataFrame(index = ['first', 'third', 'fifth'])
cnt = 1
for note in h_col:
    if 'm' in note:
        harmony = [0, 3, 7]
    else:
        harmony = [0, 4, 7]
        
    changed2 = []
    for i in harmony:
        changed = transpose_line(f'| {note} |', i, 'A')
        changed2.append(key_regex.findall(changed)[0])
    harmony_result[f'{cnt}. {note}'] = changed2
    cnt += 1

In [107]:
harmony_result

,1. C,2. Dm,3. Em,4. Bm,5. Cm,6. Fm,7. Am,8. D
first,C,D,E,B,C,F,A,D
third,E,F,G,D,D#,G#,C,F#
fifth,G,A,B,F#,G,C,E,A
